In [1]:
import numpy as np
import pandas as pd
import string
import spacy
from spacy.tokenizer import Tokenizer
import re
from nltk import tokenize
from nltk.corpus import stopwords
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from tensorflow.python.client import device_lib

Using TensorFlow backend.


In [2]:
nlp = spacy.load('pt_core_news_sm')

# ETAPA DE CARREGAMENTO DO DATASET

In [3]:
data = pd.read_json('tcc1.json')

In [4]:
data_titulo = data.título
data_noticia = data.texto

# ETAPA DE PRÉ-PROCESSAMENTO

In [5]:
noticia = []
titulo = []
for t in range(len(data_titulo)):
    titulo.append(data_titulo[t].lower())
for n in range(len(data_noticia)):
    noticia.append(data_noticia[n].lower())

In [6]:
stopwords = spacy.lang.pt.STOP_WORDS

In [7]:
pontos = []
for i in string.punctuation:
    pontos.append(i)

In [8]:
def frase_limpa(texto):
    palavra = []
    texto = nlp(texto)
    for p in texto:
        palavra.append(p.text)
    texto = [p for p in palavra if p not in stopwords]
    texto = [t for t in texto if t not in pontos]
    texto = ' '.join([i for i in texto])
    return texto

In [9]:
noticia_limpa_stopwords = [frase_limpa(n) for n in noticia]
titulo_limpa_stopwords = [frase_limpa(t) for t in titulo]

In [10]:
def remove_numeros(texto):
    texto = re.sub('[0-9!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]*', '', texto)
    return texto

In [11]:
noticia_limpa = [remove_numeros(nl) for nl in noticia_limpa_stopwords]
titulo_limpo = [remove_numeros(tl) for tl in titulo_limpa_stopwords]

# ETAPA DE TOKENIZAÇÃO

In [12]:
token_noticia = []
for nl in noticia_limpa:
    palavra = nl.split(' ')
    for p in palavra:
        if len(p) > 2:
            token_noticia.append(p)

In [13]:
token_titulo = []
for tl in titulo_limpo:
    palavra = tl.split(' ')
    for p in palavra:
        if len(p) > 2:
            token_titulo.append(p)

In [15]:
tokenizer = Tokenizer(nlp.vocab)

doc = nlp("Eu gosto de maçãs")
apples = doc[2]
print(apples.vector.dtype)
print(apples.vector.shape)
print(apples.vector)
print(apples.vector_norm)

In [17]:
token_input = [nlp(t) for t in token_noticia]

In [18]:
token_input = [t.vector_norm for t in token_input]

In [25]:
token_decode = [nlp(t) for t in token_titulo]
token_decode = [t.vector_norm for t in token_decode]

In [19]:
token_input = np.unique(token_input)

In [20]:
len(token_input)

13582

In [21]:
token_input

array([29.14407176, 31.62273511, 33.65379745, ..., 44.9824004 ,
       45.05344839, 45.30097087])

In [22]:
len(token_noticia)

73663

In [23]:
len(nlp.vocab)

10055

In [24]:
tam_max = max([len(frase)] for frase in noticia_limpa)
tam_max = tam_max[0]

In [26]:
titulo_max = max([len(titulo)] for titulo in titulo_limpo)
titulo_max = titulo_max[0]

In [28]:
frase_input = [nlp(t) for t in noticia_limpa]
frase_input = [t.vector_norm for t in frase_input]

In [29]:
titulo_input = [nlp(t) for t in titulo_limpo]
titulo_input = [t.vector_norm for t in titulo_input]

In [30]:
encode_input = tf.keras.preprocessing.sequence.pad_sequences(frase_input, value = 0, padding='post', maxlen=tam_max)
decode_input = tf.keras.preprocessing.sequence.pad_sequences(titulo_input, value = 0, padding='post', maxlen=tam_max)

ValueError: `sequences` must be a list of iterables. Found non-iterable: 16.76141438269871